In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
import torch
import torchaudio
import pickle
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel
from speechbrain.pretrained import EncoderClassifier
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# IEMOCAP dataset paths
IEMOCAP_TRAIN_PATH = "metadata/IEMOCAP_metadata_train.csv"
IEMOCAP_VAL_PATH = "metadata/IEMOCAP_metadata_val.csv"
IEMOCAP_TEST_PATH = "metadata/IEMOCAP_metadata_test.csv"


In [3]:
audio_model = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")
print("All loaded")

INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetc

All loaded


In [ ]:
# Create training set for IEMOCAP
train_list = pd.read_csv(IEMOCAP_TRAIN_PATH)
train_pkl = []
with torch.no_grad():
    for idx in range(len(train_list)):
        audio_file = train_list['audio_file'][idx]
        audio_signal, _ = torchaudio.load(audio_file, normalize=True)
        audio_outputs = audio_model.encode_batch(audio_signal)
        audio_embed = audio_outputs.mean(axis=0)[0]
        label = train_list['label'][idx]
        label = torch.tensor(label)
        train_pkl.append({
            'audio_embed': audio_embed,
            'label': label
        })

train_file = open("features/IEMOCAP_ECAPA_train.pkl", "wb")

pickle.dump(train_pkl, train_file)

train_file.close()

In [6]:
# Create validation set for IEMOCAP
val_list = pd.read_csv(IEMOCAP_VAL_PATH)
val_pkl = []
with torch.no_grad():
    for idx in range(len(val_list)):
        audio_file = val_list['audio_file'][idx]
        audio_signal, _ = torchaudio.load(audio_file, normalize=True)
        audio_outputs = audio_model.encode_batch(audio_signal)
        audio_embed = audio_outputs.mean(axis=0)[0]
        label = val_list['label'][idx]
        label = torch.tensor(label)
        val_pkl.append({
            'audio_embed': audio_embed,
            'label': label
        })
val_file = open("features/IEMOCAP_ECAPA_val.pkl", "wb")

pickle.dump(val_pkl, val_file)

val_file.close()


In [7]:
test_list = pd.read_csv(IEMOCAP_TEST_PATH)
test_pkl = []
with torch.no_grad():
    for idx in range(len(test_list)):
        audio_file = test_list['audio_file'][idx]
        audio_signal, _ = torchaudio.load(audio_file, normalize=True)
        audio_outputs = audio_model.encode_batch(audio_signal)
        audio_embed = audio_outputs.mean(axis=0)[0]
        label = test_list['label'][idx]
        label = torch.tensor(label)
        test_pkl.append({
            'audio_embed': audio_embed,
            'label': label
        })
test_file = open("features/IEMOCAP_ECAPA_test.pkl", "wb")

pickle.dump(test_pkl, test_file)

test_file.close()